In [ ]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import re
import spacy

In [ ]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 985 kB 4.8 MB/s 
     |████████████████████████████████| 358 kB 47.0 MB/s 
     |████████████████████████████████| 140 kB 40.8 MB/s 
     |████████████████████████████████| 58 kB 4.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.


In [ ]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu b

In [ ]:
wd.get("https://topostext.org/work/14")

In [ ]:
topos_test = BeautifulSoup(wd.page_source, features='html')

In [ ]:
p_tags = topos_test.find_all('p')[2:]

In [ ]:
import collections
TRAIN_DATA = []

for i, p in enumerate(p_tags):
  document = []
  document.append(p.text.replace('\xa0\xa0', ''))

  ent_dict = collections.defaultdict(list)
  for a in p.find_all('a'):
    span_list = []
    if a.text.istitle():
      if 'href' in a.attrs:
        span_list.append(a.text) 
        span_list.append(a['href'].split('/')[1])
      else:
        span_list.append(a.text)
        span_list.append(a['class'][0])
    if len(span_list) > 0:
      ent_dict['entities'].append(span_list) 
  document.append(dict(ent_dict))
  TRAIN_DATA.append(document)   

In [ ]:
for id in range(len(TRAIN_DATA)):
  test = TRAIN_DATA[id]
  if 'entities' in test[1].keys():
    ent_list = test[1]['entities']
    ent_spans = []
    for i,l in enumerate(ent_list):
      s = list(re.finditer(f'{l[0]}',test[0]))[ent_list[:i].count(l)].span()
      l = l[1:]
      l.insert(0,s[1])
      l.insert(0,s[0])
      ent_spans.append(l)
    test[1]['entities'] = ent_spans

In [ ]:
def createTRAIN_DATA(url):
  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  wd.get(url)
  soup = BeautifulSoup(wd.page_source, features='html')
  TRAIN_DATA = [] 

  p_tags = soup.find_all('p')[1:]
  for i, p in enumerate(p_tags):
    document = []
    document.append(p.text.replace('\xa0\xa0', ''))
    ent_dict = collections.defaultdict(list)
    for a in p.find_all('a'):
      span_list = []
      if a.text.istitle():
        if 'href' in a.attrs:
          span_list.append(a.text) 
          span_list.append(a['href'].split('/')[1])
        else:
          span_list.append(a.text)
          span_list.append(a['class'][0])
      if len(span_list) > 0:
        ent_dict['entities'].append(span_list) 
    document.append(dict(ent_dict))
    TRAIN_DATA.append(document)   

  for id in range(len(TRAIN_DATA)):
    test = TRAIN_DATA[id]
    if 'entities' in test[1].keys():
      ent_list = test[1]['entities']
      ent_spans = []
      for i,l in enumerate(ent_list):
        s = list(re.finditer(f'{l[0]}',test[0]))[ent_list[:i].count(l)].span()
        l = l[1:]
        l.insert(0,s[1])
        l.insert(0,s[0])
        ent_spans.append(l)
      test[1]['entities'] = ent_spans

  return TRAIN_DATA


In [ ]:
train = createTRAIN_DATA("https://topostext.org/work/5")

In [ ]:
len(train)

82

In [ ]:
!mkdir data

In [ ]:
from spacy.tokens import DocBin
from pathlib import Path
import warnings

def convert(lang: str, TRAIN_DATA, output_path: Path):
    nlp = spacy.blank(lang)
    nlp.max_length = 2000000
    db = DocBin()
    for text, annot in TRAIN_DATA:
        doc = nlp.make_doc(text)
        ents = []
        if 'entities' in annot.keys():
          for start, end, label in annot["entities"]:  
              span = doc.char_span(start, end, label=label)
              if span is None:
                  msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                  warnings.warn(msg)
              else:
                  #print(TRAIN_DATA.index([text,annot])) ## to check repeats
                  ents.append(span)
        try:
          doc.ents = ents
        except:
          continue
        db.add(doc)
    db.to_disk(output_path)

In [ ]:
TRAIN_DATA = []
for i in tqdm(range(2, 23)):
  train = createTRAIN_DATA(f'https://topostext.org/work/{i}') ## arbitrary amount
  for item in train:
    TRAIN_DATA.append(item)

100%|██████████| 21/21 [01:21<00:00,  3.90s/it]


In [ ]:
import pandas as pd
data_series = pd.Series(TRAIN_DATA)
valid_series = data_series.sample(frac=.2, random_state=1)
train = data_series.drop(valid_series.index).to_list()
valid = valid_series.to_list()

In [ ]:
convert("en", train, "/content/data/train.spacy")
convert("en", valid, "/content/data/valid.spacy")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Skipping entity [169, 175, demonym] in the following text because the character span 'Trojan' does not align with token boundaries:

"§ 2.350For I declare that Cronos' son, supreme in might, gave promise with his nod on that day when the Argives went on board their swift-faring ships, bearing unto the Trojans death and fate; for he lightened on our right and shewed forth signs of good. Wherefore let no man make haste to depart homewards until each have lain with the wife of some Trojan, "

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Skipping entity [7, 13, people] in the following text because the character span 'Nestor' does not align with token boundaries:

'§ 2.405Nestor, first of all, and king Idomeneus, and thereafter the twain Aiantes and the son of Tydeus, and as the sixth Odysseus, the peer of Zeus in counsel. And unbidden came to him Menelaus, good at the war-cry, fo

In [ ]:
!python -m spacy init fill-config /content/base_config.cfg /content/config.cfg

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2022-09-17 21:24:17.787523: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Auto-filled config with all values
✔ Saved config
/content/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
from spacy.cli.train import train

In [ ]:
!mkdir models

In [ ]:
!python -m spacy debug data /content/config.cfg

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
2022-09-17 21:24:24.531720: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected

============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
6527 training docs
1632 evaluation docs
✔ No overlap between training and evaluation data

============================== Vocab & Vectors ==============================
ℹ 657548 total word(s) in the data (25176 unique)
ℹ No word vectors present in the package

========================== Named Entity Recognition ==========================
ℹ 5 label(s)
0 missing value(s) (tokens 

In [ ]:
train("/content/config.cfg",
      overrides={"paths.train": "/content/data/train.spacy",
                 "paths.dev": "/content/data/valid.spacy"},
    output_path="models/")

ℹ Saving to output directory: models
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     38.91    0.00    0.00    0.00    0.00
  0     200         43.65   1596.85   72.05   73.90   70.30    0.72
  0     400         93.96    481.93   78.35   77.16   79.58    0.78
  0     600        114.11    569.38   83.08   80.43   85.91    0.83
  0     800        153.24    451.30   85.74   84.99   86.51    0.86
  0    1000        182.77    569.75   87.85   85.03   90.86    0.88
  0    1200        193.46    502.28   85.21   86.75   83.72    0.85
  0    1400        269.59    527.28   89.71   89.19   90.25    0.90
  0    1600        366.12    63

In [ ]:
inf_text ='''
An upper vest, once Helen's rich attire,
From Argos by the fam'd adultress brought,
With golden flow'rs and winding foliage wrought,
Her mother Leda's present, when she came
To ruin Troy and set the world on flame;
The scepter Priam's eldest daughter bore,
Her orient necklace, and the crown she wore
Of double texture, glorious to behold,
One order set with gems, and one with gold.
'''

In [ ]:
trained_nlp = spacy.load("/content/models/model-best")

In [ ]:
doc = trained_nlp(inf_text)

In [ ]:
for ent in doc.ents:
  print(ent, ent.label_)

Helen people
Argos place
Leda people
Troy place
Priam people
